In [1]:
# Import required packages: 
import pandas as pd

In [2]:
import ee
# Required if you don't have an environment variable setup with the Authentication Key:
# ee.Authenticate()
ee.Initialize()

# Spatially thin locations and export to asset

## Helper Functions and imports

In [33]:
# HELPER FUNCTIONS
#//========================================================

def filterDistance(points, distance):
    # Spatial thinning function
    def iter_func(el, ini):
        ini = ee.List(ini)
        fcini = ee.FeatureCollection(ini)
        buf = ee.Feature(el).geometry().buffer(distance)
        s = fcini.filterBounds(buf).size()
        cond = s.lte(0)
        return ee.Algorithms.If(cond, ini.add(el), ini)
    filt2 = ee.List([])
    filt = points.iterate(iter_func, filt2)
    filtered = ee.FeatureCollection(ee.List(filt))
    return filtered

#//========================================================




In [34]:
# Instantiate list so that thinning algorithm can be performed in a spatially explicit manner
s_dates = ee.List([
ee.Date('2001-01-01'),ee.Date('2002-01-01'),
ee.Date('2003-01-01'),ee.Date('2004-01-01'),
ee.Date('2005-01-01'),ee.Date('2006-01-01'),
ee.Date('2007-01-01'),ee.Date('2008-01-01'),
ee.Date('2009-01-01'),ee.Date('2010-01-01'),
ee.Date('2011-01-01'),ee.Date('2012-01-01'),
ee.Date('2013-01-01'),ee.Date('2015-01-01'),
ee.Date('2016-01-01'),ee.Date('2017-01-01')
])

In [ ]:
# Function that filters the feature collection and spatially thins it
def filter_date_space(date):
    start_date = ee.Date(date).advance(1,'year')
    end_date = start_date.advance(1,'year')
    points_in_that_year = coll.filterDate(start_date, end_date)
    
    spatially_filt = filterDistance(points_in_that_year, distance)
    
    return spatially_filt

In [35]:
carp_og = ee.FeatureCollection("users/seancliffcarter/PresenceData/Asian_Carp"),
quagga_og = ee.FeatureCollection("users/seancliffcarter/PresenceData/Quagga_clean"),
hydrilla_og = ee.FeatureCollection("users/seancliffcarter/PresenceData/hydrilla_clean"),
zebra_og = ee.FeatureCollection("users/seancliffcarter/zebra_clean"),
snakehead_og = ee.FeatureCollection("users/seancliffcarter/snakehead_clean");

coll = ee.FeatureCollection(zebra_og) # <----- CHANGE NAME FOR DIFFERENT DATASETS
distance = 5000

## Perform spatial thinning algorithm and export to asset

In [ ]:
feats = s_dates.map(filter_date_space)
def merge_coll(first_year, second_year):
    return ee.FeatureCollection(first_year).merge(ee.FeatureCollection(second_year))

first = ee.FeatureCollection([])
spatially_thin = ee.FeatureCollection(feats.iterate(merge_coll, first))

In [ ]:
export = ee.batch.Export.table.toAsset(collection = spatially_thin,
                    description = 'zebra_spatially_thinned', # <-------- CHANGE NAME FOR DIFFERENT DATA
                    assetId = 'users/seancliffcarter/spatially_thinned/zebra')# <----- CHANGE NAME
export.start()

# Build Big Raster Image

## Import assets

In [20]:
# MODIS Mission
modusGlobal = ee.ImageCollection("MODIS/006/MYD11A2")

# Primary Productivity
GPP = ee.ImageCollection("UMT/NTSG/v2/LANDSAT/GPP")

# Surface water
pikelSurfaceWater = ee.Image("JRC/GSW1_1/GlobalSurfaceWater")

# Elevation
DEM = ee.Image("USGS/NED")

# Enhanced Vegetation Index and NDVI
modusVeg = ee.ImageCollection("MODIS/006/MYD13A2")

# Heat Isolation Load
CHILI = ee.Image("CSP/ERGo/1_0/Global/SRTM_CHILI")

# Topographic Diversity
topoDiversity = ee.Image("CSP/ERGo/1_0/Global/ALOS_topoDiversity")

# Vegetation Continuous Field product - percent tree cover, etc
VCF = ee.ImageCollection("MODIS/006/MOD44B")

# Human Modification index
gHM = ee.ImageCollection("CSP/HM/GlobalHumanModification")


# Climate information
NLDAS = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")

# Shape file containing Country Boundaries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Dynamic Surface Water metric
pekel_monthly_water = ee.ImageCollection("JRC/GSW1_2/MonthlyHistory")

# Static surface water metric
pekel_static_water = ee.ImageCollection('JRC/GSW1_2/MonthlyRecurrence')

# SPATIALLY THINNED FEATURE COLLECTIONS- Exported from above script
zebra = ee.FeatureCollection("users/seancliffcarter/PresenceData/zebra_spatially_thinned")
carp = ee.FeatureCollection("users/seancliffcarter/PresenceData/carp_spatially_thinned")
quagga = ee.FeatureCollection("users/seancliffcarter/PresenceData/quagga_spatially_thinned")
snake = ee.FeatureCollection("users/seancliffcarter/PresenceData/snake_spatially_thinned")
hydrilla = ee.FeatureCollection("users/seancliffcarter/hydrilla_spatiallythinned")


# Hydrilla Geometry (drawn manually)
geometry = ee.Geometry.Polygon(
        [[[-100.95683057308197, 47.53111193980715],
          [-100.95683057308197, 25.104255042806013],
          [-68.43729932308199, 25.104255042806013],
          [-68.43729932308199, 47.53111193980715]]], None, False)

# Helper function to embedd ee.FeatureCollections with EE readible date
def embedd_date(x):
    year = ee.Number(x.get("year")).floor()
    ee_date = ee.Date.fromYMD(year, 1, 15)
    return x.set("system:time_start", ee_date)

# zebra = zebra.map(embedd_date)
# carp = carp.map(embedd_date)
# quagga = quagga.map(embedd_date)
# snake = snake.map(embedd_date)
hydrilla = hydrilla.map(embedd_date)

## Select features, etc


In [21]:
#========================================================
#Rename Bands and select bands, etc
#========================================================


NLDAS_precip = NLDAS.select("total_precipitation");
NLDAS_temp = NLDAS.select("temperature");
NLDAS_humid = NLDAS.select("specific_humidity");
NLDAS_potEvap = NLDAS.select("potential_evaporation");


CHILI = CHILI.rename(['Heat_Insolation_Load'])
srtmChili = CHILI.select('Heat_Insolation_Load');
topoDiversity = topoDiversity.rename(["Topographic_Diversity"])
topoDiv = topoDiversity.select("Topographic_Diversity")
footprint = ee.Image(gHM.first().select("gHM"));

# Surface water occurrence
sw_occurrence = pekel_static_water\
                      .select('monthly_recurrence')\
                      .mean()\
                      .rename(['SurfaceWaterOccurrence'])\
                      .unmask()


## Mask features by quality control bands

In [22]:
# ========================================================
# Masking TPP via quality control bands
# ========================================================
def gpp_qc(img):
    img2 = img.rename(['GPP','QC']);
    quality = img2.select("QC")
    mask = quality.neq(11) \
                .And(quality.neq(10)) \
                .And(quality.neq(20)) \
                .And(quality.neq(21)) 
    return img2.mask(mask)

GPP_QC = GPP.map(gpp_qc);

# ========================================================
# Masking LST via quality control bands
# ========================================================
def lst_qc(img):
    quality = img.select("QC_Day")
    mask = quality.bitwiseAnd(3).eq(0) \
                .And(quality.bitwiseAnd(12).eq(0))
    return img.mask(mask)

LST = modusGlobal.map(lst_qc) \
                 .select("LST_Day_1km");

        
# ========================================================
# Mask Modus Vegetation Indices by quality flag
# ========================================================
def modusQC(image):
    quality = image.select("SummaryQA")
    mask = quality.eq(0)
    return image.updateMask(mask)

modusVeg_QC = modusVeg.map(modusQC)
EVI = modusVeg_QC.select("EVI")
NDVI = modusVeg_QC.select("NDVI")

# ========================================================
# Mask Continuous Fields via quality control band
# ========================================================
def VCFqc(img):
    quality = img.select("Quality")
    mask = quality.bitwiseAnd(2).eq(0) \
                    .And(quality.bitwiseAnd(4).eq(0)) \
                    .And(quality.bitwiseAnd(8).eq(0)) \
                    .And(quality.bitwiseAnd(16).eq(0)) \
                    .And(quality.bitwiseAnd(32).eq(0))

    return img.mask(quality)

VCF_qc = VCF.map(VCFqc)


## Define helper filters and lists to iterate over

In [23]:

#========================================================
# Define convex hull from which to draw pseudo absences
#========================================================
# Define CONUS
CONUS = countries.filter(ee.Filter.eq("country_co","US")) \
                  .filter(ee.Filter.eq("country_na","United States"))

zebra_hull = zebra.geometry().convexHull().intersection(CONUS)
carp_hull = carp.geometry().convexHull().intersection(CONUS)
quagga_hull = quagga.geometry().convexHull().intersection(CONUS)
snake_hull = snake.geometry().convexHull().intersection(CONUS)
hydrilla_hull = hydrilla.geometry().convexHull().intersection(CONUS)

#========================================================


#========================================================
# Build Lists from which to map over
#========================================================

# List from which absences will be built
ee_dates = ee.List([
    ee.Date('2001-01-01'),ee.Date('2002-01-01'),
    ee.Date('2003-01-01'),ee.Date('2004-01-01'),
    ee.Date('2005-01-01'),ee.Date('2006-01-01'),
    ee.Date('2007-01-01'),ee.Date('2008-01-01'),
    ee.Date('2009-01-01'),ee.Date('2010-01-01'),
    ee.Date('2011-01-01'),ee.Date('2012-01-01'),
    ee.Date('2013-01-01'),ee.Date('2015-01-01'),
    ee.Date('2016-01-01'),ee.Date('2017-01-01')])

# Required for exporting presence locations due to covariate availability 
ee_dates_presence = ee.List([
    ee.Date('2001-01-01'),ee.Date('2002-01-01'),
    ee.Date('2003-01-01'),ee.Date('2004-01-01'),
    ee.Date('2005-01-01'),ee.Date('2006-01-01'),
    ee.Date('2007-01-01'),ee.Date('2008-01-01'),
    ee.Date('2009-01-01'),ee.Date('2010-01-01'),
    ee.Date('2011-01-01'),ee.Date('2012-01-01'),
    ee.Date('2013-01-01'),ee.Date('2015-01-01'),
    ee.Date('2016-01-01')])

## Annual Cube function

In [24]:
#========================================================
# "Builder Function" -- processes each annual variable into a list of images
#========================================================

def build_annual_cube(d):
    # Set start and end dates for filtering time dependent predictors (SR, NDVI, Phenology)
      # Advance startDate by 1 to begin with to account for water year (below)
    startDate = (ee.Date(d).advance(1.0,'year').millis())
    endDate = ee.Date(d).advance(2.0,'year').millis()

  #========================================================
  #Define function to compute seasonal information for a given variable
  #========================================================
    def add_seasonal_info(imgCol,name,bandName):
        winter = imgCol.filterDate(winter_start,winter_end)
        spring = imgCol.filterDate(spring_start,spring_end)
        summer = imgCol.filterDate(summer_start,summer_end)
        fall = imgCol.filterDate(fall_start,fall_end)

        winter_tot = winter.sum()
        spring_tot = spring.sum()
        summer_tot = summer.sum()
        fall_tot = fall.sum()

        winter_max = winter.max()
        winter_min = winter.min()
        spring_max = spring.max()
        spring_min = spring.min()
        summer_max = summer.max()
        summer_min = summer.min()
        fall_max = fall.max()
        fall_min = fall.min()

        winter_diff = winter_max.subtract(winter_min)
        spring_diff = spring_max.subtract(spring_min)
        summer_diff = summer_max.subtract(summer_min)
        fall_diff = fall_max.subtract(fall_min)

        names = ['winter_total'+name,'spring_total'+name,'summer_total'+name,
                      'fall_total'+name]

        return winter_tot.addBands([spring_tot,summer_tot,fall_tot]) \
                         .rename(names)

  # Set up Seasonal dates for precip, seasonal predictors
    winter_start = ee.Date(startDate)
    winter_end = ee.Date(startDate).advance(3,'month')
    spring_start = ee.Date(startDate).advance(3,'month')
    spring_end = ee.Date(startDate).advance(6,'month')
    summer_start = ee.Date(startDate).advance(6,'month')
    summer_end = ee.Date(startDate).advance(9,'month')
    fall_start = ee.Date(startDate).advance(9,'month')
    fall_end = ee.Date(endDate)

  # Aggregate seasonal info for each variable of interest (potEvap neglected purposefully)
    seasonal_precip = add_seasonal_info(NLDAS_precip,"Precip","total_precipitation")
    seasonal_temp = add_seasonal_info(NLDAS_temp,"Temp","temperature")
    seasonal_humid = add_seasonal_info(NLDAS_humid,"Humidity","specific_humidity")

    waterYear_start = ee.Date(startDate).advance(10,'month')
    waterYear_end = waterYear_start.advance(1,'year')

  #========================================================
  # Aggregate Other Covariates
  #========================================================

  # Vegetative Continuous Fields
    meanVCF = VCF_qc.filterDate(startDate, endDate) \
                      .mean()

  # Filter Precip by water year to get total precip annually

    waterYearTot = NLDAS_precip.filterDate(waterYear_start,waterYear_end) \
                                 .sum()

  # Find mean EVI per year:
    maxEVI = EVI.filterDate(startDate,endDate) \
                  .mean() \
                  .rename(['Mean_EVI'])

  #Find mean NDVI per year:
    maxNDVI = NDVI.filterDate(startDate,endDate) \
                    .mean() \
                    .rename(["Mean_NDVI"])

  # Find flashiness per year by taking a Per-pixel Standard Deviation:
    flashiness_yearly = ee.Image(pekel_monthly_water.filterDate(startDate,endDate) \
                                                      .reduce(ee.Reducer.sampleStdDev()) \
                                                      .select(["water_stdDev"])) \
                                                      .rename("Flashiness")

  # Find max LST per year:
    maxLST = LST.max().rename(["Max_LST_Annual"])

  # Find mean GPP per year:
    maxGPP = GPP_QC.filterDate(startDate,endDate) \
                      .mean() \
                      .rename(['Mean_GPP','QC'])

  # All banded images that don't change over time
    static_input_bands = sw_occurrence.addBands(DEM.select("elevation")) \
                                          .addBands(srtmChili) \
                                          .addBands(topoDiv) \
                                          .addBands(footprint)

  # Construct huge banded image
    banded_image = static_input_bands \
                          .addBands(srcImg = maxLST, names = ["Max_LST_Annual"]) \
                          .addBands(srcImg = maxGPP, names = ["Mean_GPP"]) \
                          .addBands(srcImg =  maxNDVI, names = ["Mean_NDVI"]) \
                          .addBands(srcImg = maxEVI, names = ["Mean_EVI"]) \
                          .addBands(meanVCF.select("Percent_Tree_Cover")) \
                          .addBands(seasonal_precip) \
                          .addBands(flashiness_yearly) \
                          .set("system:time_start",startDate)

    return banded_image




In [25]:

#========================================================
# Run covariate algorithm and build a huge image
# with one band corresponding to each year / covariate
#========================================================

# Image Collection
banded_images = ee.ImageCollection(ee_dates.map(build_annual_cube))

def format_date(image):
    dateFormatted = ee.String('_').cat(image.date().format('YYYY'))
    return image.rename(image.bandNames().map(lambda x: ee.String(x).cat(dateFormatted)))

    
# Rename Image Collection so that each image is named by year
renamed_collection = banded_images.map(format_date)


# Squash big image collection down to one (LARGE!) image
big_img = renamed_collection.toBands().regexpRename('^(.*)', 'b_$1')





# PseudoAbsence Selection

In [ ]:
#========================================================
# Sample locations at each presence / "absence" location
#========================================================
# Helper function to include "Presence" Column
def embedd_presence(feat):
    return feat.set("Present", 0)

count = 0
while count < 300:
    # Sample from large image, with a different set of points for each "count"
    absence_samp = big_img.sample(**{'region':zebra_hull, # <-------- CHANGE HULL TO SPECIES
                                'scale':30,
                                'numPixels':20,
                                'seed': count,
                                'dropNulls':False}).map(embedd_presence)

    # Convert EE object to python JSON
    absence_json = absence_samp.getInfo()
    
    # Get each feature in the sample
    list_of_features = absence_json['features']
    
    # List comprehension to receive columns ("propreties") for each feature
    new_data = [x['properties'] for x in list_of_features]

    # Write directly to hard drive
    pd.DataFrame(new_data).to_csv("Zebra/Absences/another_100_"+str(count)+".csv",index=False) # < ----- CHANGE CSV NAME
    
    # Iterate so that we draw a different sample in next pass through
    count += 1



# Sample image collection with temporally explicit presence information

In [26]:
# Instead of sampling from the large image, we must sample from a temporally filtered image.
banded_images = ee.ImageCollection(ee_dates_presence.map(build_annual_cube))


def sample_from_collection(d):
    # Set start and end dates for filtering time dependent predictors (SR, NDVI, Phenology)
      # Advance startDate by 1 to begin with to account for water year (below)
    startDate = (ee.Date(d).advance(1.0,'year').millis())
    endDate = ee.Date(d).advance(2.0,'year').millis()
    
     # Filter points by dates
    pointsInThatYear = zebra.filterDate(startDate, endDate) # <---- CHANGE ACCORDING TO SPECIES OF INTEREST
    
    # Filter collection by that year
    img_in_that_year = ee.Image(banded_images.filterDate(startDate,endDate).first())
    
    
    # Sample that image
    presence_samp = img_in_that_year.unmask().sampleRegions(**{
                                'collection':pointsInThatYear,
                                'scale': 30,
                                'properties':['Present'],
                                'tileScale':16
                              })
    
    return presence_samp

# Return Feature Collection which can be exported
presence_sample = ee.FeatureCollection(ee_dates_presence.map(sample_from_collection)).flatten()





## Filter and output flattened feature collection

In [27]:
# Embedd Feature collection with random column to split it up
embedded = presence_sample.randomColumn() #  <- MAKE SURE FEATURE COLLECTION IS CORRECT

# Define a sequence of numbers (0,49)
numpy_int_seq = np.arange(0,49,1)

# Define sequence of decimals from which we can filter (0,0.02,0.04,...0.98)
numpy_rand_seq = np.arange(0,1,0.02)

def filter_and_output_features(s):
    
    # Greater than and less than values from which to filter
    gt_val = ee.Number(numpy_rand_seq[s])
    lt_val = ee.Number(numpy_rand_seq[s+1])
    
    # Instantiate filters
    filt = ee.Filter.rangeContains('random', gt_val, lt_val)
    filtered = embedded.filter(filt)
    
    
    # Perform python-fu to export dataframe
    presence_json = filtered.getInfo()

    list_of_features = presence_json['features']

    new_data = [x['properties'] for x in list_of_features]
    
    pd.DataFrame(new_data).to_csv("Zebra/presence_zebra_TEST_" + str(s) + ".csv",index=False) # <--------- #CHANGE OUTPUT CSV

In [ ]:
# Loop through all of the random column's slices
for s in numpy_int_seq:
    filter_and_output_features(s)